In [1]:

%pprint

Pretty printing has been turned OFF


In [16]:

%matplotlib inline
from IPython.display import clear_output, display
from PIL import Image
from cohere.error import CohereAPIError
from datetime import datetime
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere, OpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter, SpacyTextSplitter
from langchain.vectorstores import Chroma
from neo4j.exceptions import ServiceUnavailable
from openai.error import RateLimitError
from pandas import DataFrame
from ratelimit import limits, sleep_and_retry
from stability_sdk import client as stability_client
from tqdm.notebook import tqdm
import cohere
import getpass
import humanize
import io
import os
import pandas as pd
import random
import re
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as stability_generation
import sys
import time
import warnings
import winsound

warnings.filterwarnings('ignore')
duration = 1000  # milliseconds
freq = 880  # Hz

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

# Get the Storage object
from storage import Storage
s = Storage(
    data_folder_path=os.path.abspath('../data'),
    saves_folder_path=os.path.abspath('../saves')
)

# Get the WebScrapingUtilities object
from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(
    s=s,
    secrets_json_path=os.path.abspath('../data/secrets/jh_secrets.json')
)

os.environ['SERPAPI_API_KEY'] = wsu.secrets_json['SERPAPI_API_KEY']

# Paste your API key here. Remember to not share it publicly
co_key = wsu.secrets_json['Cohere_API_Key']
os.environ['COHERE_API_KEY'] = co_key
co = cohere.Client(co_key)

# To get your API key, visit https://beta.dreamstudio.ai/membership
os.environ['STABILITY_KEY'] = wsu.secrets_json['Dream_Studio_API_Key']
stability_api = stability_client.StabilityInference(
    key=os.environ['STABILITY_KEY'], 
    verbose=True,
)

In [4]:

# Get the HeaderAnalysis object
from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(s=s, verbose=False)

# Get the CypherUtilities object and Neo4j driver
from cypher_utils import CypherUtilities
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']
cu = CypherUtilities(
    uri=uri, user=user, password=password, driver=None, s=s, ha=ha
)

# Get the SectionLRClassifierUtilities object
from section_classifier_utils import SectionLRClassifierUtilities
slrcu = SectionLRClassifierUtilities(ha=ha, cu=cu, verbose=False)

# Check if the slrcu has built its parts-of-speech logistic regression elements
t1 = time.time()
if not hasattr(slrcu, 'pos_predict_percent_fit_dict'):
    slrcu.build_pos_logistic_regression_elements(sampling_strategy_limit=None, verbose=True)
if hasattr(slrcu, 'pos_predict_percent_fit_dict'):
    print('predict_single is now available')
else:
    print('predict_single is not available')
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Parts-of-speech logistic regression elements built in {duration_str}')

I have 49,163 labeled parts of speech in here
predict_single is now available
Parts-of-speech logistic regression elements built in 9 seconds


In [5]:

# Get the SectionCRFClassifierUtilities object
from section_classifier_utils import SectionCRFClassifierUtilities
scrfcu = SectionCRFClassifierUtilities(cu=cu, ha=ha, verbose=False)

# Check if the scrfcu has built its parts-of-speech conditional random field elements
t1 = time.time()
if not hasattr(scrfcu, 'pos_symbol_crf'):
    scrfcu.build_pos_conditional_random_field_elements(verbose=True)
if hasattr(scrfcu, 'pos_predict_percent_fit_dict'):
    print('predict_single is now available')
else:
    print('predict_single is not available')
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Parts-of-speech conditional random field elements built in {duration_str}')

predict_single is now available
Parts-of-speech conditional random field elements built in 1 second


In [6]:

# Get the SectionSGDClassifierUtilities object
from section_classifier_utils import SectionSGDClassifierUtilities
ssgdcu = SectionSGDClassifierUtilities(ha=ha, cu=cu, verbose=False)

# Check if the ssgdcu has built its parts-of-speech stochastic gradient decent elements
t1 = time.time()
if not hasattr(ssgdcu, 'pos_predict_percent_fit_dict'):
    ssgdcu.build_pos_stochastic_gradient_descent_elements(sampling_strategy_limit=None, verbose=True)
if hasattr(ssgdcu, 'pos_predict_percent_fit_dict'):
    print('predict_single is now available')
else:
    print('predict_single is not available')
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Parts-of-speech stochastic gradient descent elements built in {duration_str}')

I have 49,163 labeled parts of speech in here
predict_single is now available
Parts-of-speech stochastic gradient descent elements built in 11 seconds


In [7]:

def combine_adjacent(split_strs_list):
    combined_list = []
    for i, s in enumerate(split_strs_list):
        if i == 0:
            combined_list.append(s)
        elif combined_list[-1].lower().endswith(' and'):
            combined_list[-1] = combined_list[-1] + ' ' + s
        else:
            combined_list.append(s)
    
    return combined_list

In [20]:

# Define the Trial key rate limit - 5 API calls / minute
@sleep_and_retry
@limits(calls=5, period=60)
def get_suggestion(chain, navigable_parent):
    llm_suggestion = chain.run(navigable_parent).strip()
    
    return llm_suggestion


----

In [ ]:

print(llm('Tell me a joke'))

In [ ]:

template = '''
I want you to act as a classical music historian that knows the history of a composition when given the name of it.

Here are some examples of historical descriptions:

The Symphony No. 3 in D minor by Gustav Mahler was written in sketch beginning in 1893, composed primarily in 1895,'''
template += " and took final form in 1896. Consisting of six movements, it is Mahler's longest composition and is the"
template += ' longest symphony in the standard repertoire, with a typical performance lasting around 95 to 110 minutes.'
template += ' It was voted one of the ten greatest symphonies of all time in a survey of conductors carried out by'
template += ''' the BBC Music Magazine.

The answer should detailed as if written by an expert.

What is the history of {composition}?
'''

prompt = PromptTemplate(
    input_variables=['composition'],
    template=template,
)

In [ ]:

print(prompt.format(composition='colorful socks'))

In [ ]:

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:

# Run the chain only specifying the input variable
print(chain.run('Alpine Symphony'))

In [ ]:

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=['product'],
    template='What is a good name for a company that makes {product}?',
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run('colorful socks').strip())

In [ ]:

from langchain.agents import load_tools
from langchain.agents import initialize_agent

# First, let's load the language model we're going to use to control the agent
llm = OpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(['serpapi', 'llm-math'], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)

# Now let's test it out!
agent.run('What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?')

In [ ]:

from langchain import ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input='Hi there!')

In [ ]:

conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)

In [ ]:

chat([HumanMessage(content='Translate this sentence from English to French. I love programming.')])

In [ ]:

messages = [
    SystemMessage(content='You are a helpful assistant that translates English to French.'),
    HumanMessage(content='Translate this sentence from English to French. I love programming.')
]
chat(messages)

In [ ]:

batch_messages = [
    [
        SystemMessage(content='You are a helpful assistant that translates English to French.'),
        HumanMessage(content='Translate this sentence from English to French. I love programming.')
    ],
    [
        SystemMessage(content='You are a helpful assistant that translates English to French.'),
        HumanMessage(content='Translate this sentence from English to French. I love artificial intelligence.')
    ],
]
result = chat.generate(batch_messages)
result

In [ ]:

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = 'You are a helpful assistant that translates {input_language} to {output_language}.'
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template='{text}'
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language='English', output_language='Navajo', text='I love programming.').to_messages())

In [ ]:

chat = ChatOpenAI(temperature=0)

template='You are a helpful assistant that translates {input_language} to {output_language}.'
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template='{text}'
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language='English', output_language='Navajo', text='I love programming.')

In [ ]:

# First, let's load the language model we're going to use to control the agent
chat = ChatOpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(['serpapi', 'llm-math'], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent='chat-zero-shot-react-description', verbose=True)

# Now let's test it out!
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")

In [ ]:

from langchain.prompts import MessagesPlaceholder
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

template_str = 'The following is a friendly conversation between a human and an AI. The AI is talkative and provides'
template_str += ' lots of specific details from its context. If the AI does not know the answer to a question,'
template_str += ' it truthfully says it does not know.'
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(template_str),
    MessagesPlaceholder(variable_name='history'),
    HumanMessagePromptTemplate.from_template('{input}')
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input='Hi there!')

In [ ]:

conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

In [ ]:

conversation.predict(input='Tell me about yourself.')

In [ ]:

from langchain.utilities import PythonREPL

python_repl = PythonREPL()
print(python_repl.run("""from langchain.utilities import PythonREPL;pr=PythonREPL();print(pr.run('print(1+1)'))""").strip())

In [ ]:

import textwrap

llm = OpenAI(model_name='text-davinci-003', temperature=0)
print('\n'.join(textwrap.wrap(llm('What is LangChain?').strip())))

In [ ]:

toplevel = 'https://langchain.readthedocs.io/en/latest'
soup = wsu.get_page_soup(toplevel)
print(soup.prettify())

In [ ]:

anchors_attrs = [anchor.attrs for anchor in soup.find_all('a')]

In [ ]:

paths = []
for anchor_attrs in anchors_attrs:
    try:
        classes = anchor_attrs['class']
        link = anchor_attrs['href']
        if 'reference' in classes:
            if 'internal' in classes:
                paths.append(link)
            elif 'external' in classes:
                if link.startswith('./'):
                    paths.append(link[len('./'):])
                else:
                    pass # Not a link to docs
            else:
                pass # I didn't understand that reference
        else:
            pass # Not a reference
    except KeyError:
        print('No classes or href:', anchor_attrs)

In [ ]:

import requests

paths = ['index.html'] + paths
pages = []
for path in paths:
    try:
        url = '/'.join([toplevel, path])
        resp = requests.get(url)
        resp.raise_for_status()
    except Exception:
        print(url)
    finally:
        pages.append({'content': resp.content, 'url': url})

In [ ]:

import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:

from unstructured.partition.html import partition_html

parsed_docs = [partition_html(text=page['content']) for page in pages]

In [ ]:

texts = []
for doc in parsed_docs:
    texts.append('\n\n'.join(
        [str(el).strip() for el in doc]
    ).strip().replace('\\n', '').replace(r'\xe2\x80\x99', "'"))

In [ ]:

print(*textwrap.wrap(texts[0]), sep='\n')

In [ ]:

for page, text in zip(pages, texts):
    page['text'] = text

In [ ]:

pages[0].keys()

In [ ]:

import pandas as pd

pd.DataFrame(pages).sample(10)


## Chunk the text for use inside LLM prompts

In [ ]:

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1024, chunk_overlap=128, separator=' '
)
documents = text_splitter.create_documents(
    [page['text'] for page in pages], metadatas=[{'source': page['url']} for page in pages]
)
print(documents[0].metadata['source'], *textwrap.wrap(documents[0].page_content), sep='\n')

In [ ]:

from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:

from langchain.vectorstores import FAISS

docsearch = FAISS.from_documents(documents, embeddings)

In [ ]:

from langchain.chains.qa_with_sources import load_qa_with_sources_chain

chain = load_qa_with_sources_chain(llm, chain_type='stuff')

In [ ]:

import langchain

dir(langchain.chains)

In [ ]:

from langchain.chains import sequential

dir(sequential)

In [ ]:

query = 'What is LangChain?'
docs = docsearch.similarity_search(query)
result = chain({'input_documents': docs, 'question': query})
text = '\n'.join(textwrap.wrap(result['output_text']))
text = '\n\nSOURCES:\n'.join(map(lambda s: s.strip(), text.split('SOURCES:')))
print(text)

In [ ]:

for doc in docs:
    print()
    print(doc)

In [ ]:

print(chain.llm_chain.prompt.template)

In [ ]:

assert scrfcu.pos_predict_percent_fit_dict['O-RQ']('*') == 0.0, "You need to rerun this"

In [ ]:

t1 = time.time()
cu.populate_pos_relationships(verbose=False)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Parts-of-speech relationships repopulated in {duration_str}')

In [ ]:

idx = split_orqs_df[mask_series].sort_values('orq_score').head(1).index[0]
child_str = split_orqs_df.iloc[idx].tag_name + split_orqs_df.iloc[idx].split_str + split_orqs_df.iloc[idx].tag_name.replace('<', '</')
child_str

In [ ]:

def do_cypher_tx(tx, navigable_parent, verbose=False):
    cypher_str = '''
        MATCH (np:NavigableParents)
        WHERE np.navigable_parent CONTAINS $navigable_parent
        SET
            np.is_header = 'False',
            np.is_task_scope = 'True',
            np.is_minimum_qualification = 'False',
            np.is_preferred_qualification = 'False',
            np.is_educational_requirement = 'False',
            np.is_legal_notification = 'False',
            np.is_other = 'False',
            np.is_corporate_scope = 'False',
            np.is_job_title = 'False',
            np.is_office_location = 'False',
            np.is_job_duration = 'False',
            np.is_supplemental_pay = 'False',
            np.is_interview_procedure = 'False',
            np.is_posting_date = 'False'
        ''' + cu.return_everything_str + ';'
    results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})
    
    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str, verbose=False)
row_objs_list

In [ ]:

child_str

In [ ]:

template = 'Given the following HTML strings, create a MINIMUM REQUIREMENTS section which parses out only the minimum requirements.\n\n'
template += 'HTML STRING: "'
template += html_strs_list[0]
template += '"\n=========\n'
template += 'MINIMUM REQUIREMENTS:'
template += "\nMust possess a Bachelor's degree or foreign equivalent in Statistics, Mathematics, or a closely related field"
template += "\nAcademic coursework or professional experience must include descriptive and predictive statistics"
template += "\nAcademic coursework or professional experience must include experience working with statistical programming tools,"
template += " such as R or Python"
template += "\nAcademic coursework or professional experience must include experience with Jupyter Notebooks, Matplotlib, Seaborn,"
template += " Dask, Docker, or BigQuery"
template += "\nAcademic coursework or professional experience must include experience with Google Cloud Platform"
template += "\nAcademic coursework or professional experience must include development and deployment of code (version control and CI/CD)"
template += "\nBachelor's / Master's degree in computer science or equivalent\n"
template += "\n2+ years of experience as a data analyst / engineer in a data focused environment and role\n"
template += "\nKnowledge of best practices with data wrangling/mapping/cleansing,"
template += " and other state-of-the-art relevant computational techniques and processes"
template += "\nStrong Python data science skill set"
template += "\nStrong SQL skills"
template += "\nStrong knowledge of engineering practices for development and deployment of code (version control + CI/CD)"
template += "\nSolid foundational knowledge of descriptive and predictive statistics"
template += "\nPassionate about data quality and able to clearly articulate its importance to internal and external partners in specific"
template += " contexts"
template += "\nCollaborative self-starter"
template += "\nClear and strong written and verbal communication skills"
template += "\nClear and strong presentation skills"
template += "\nClear and strong documentation (technical and non-technical) skills"
template += "\nAbility to collaborate with other peers and senior / principal engineers"
template += "\nEffective time management based on priorities dictated by the business"
template += "\nSelf-assessment and situational assessment skills"
template += "\nService orientation toward customers (both internal and external)"
template += "\nResilient in the face of adversity"
template += "\nAbility to solve technical and business problems with help from other team members collaboratively"
template += "\n\nHTML STRING: " + html_strs_list[1] + "\n=========\nMINIMUM REQUIREMENTS:"
template += "\nBachelors degree in Computer Science, Engineering (any) or a related field"
template += "\nOne (1) year of related work experience involving working across all phases of the Agile methodology"
template += " project delivery lifecycle for designing, developing, testing and implementing big data solutions,"
template += " data analytics, data visualization, data management, and OFSAA solutions for enterprise-level clients"
template += "\nOne (1) year of related work experience involving interacting directly with stakeholders in gathering"
template += " functional and technical requirements, analyzing client requirements, and translating requirements"
template += " into project designs to deliver solutions that satisfy business needs using Confluence wiki, Atlassian"
template += " Jira, Git, Microsoft (MS) Powerpoint, Visio And Sharepoint"
template += "\nOne (1) year of related work experience involving utilizing Cloudera Hadoop, Confluent KAFKA, OFSAA,"
template += " Arcadia Data, Unix, procedural language (PL/SQL) and SQL for the design and development of data solutions"
template += "\nOne (1) year of related work experience involving tracking and reporting on project and milestone"
template += " deliverable status to ensure timely project delivery and communicating updates to internal and external"
template += " stakeholders using Atlassian Jira, Confluence, Sharepoint, Powerpoint And Visio"
template += "\nOne (1) year of related work experience involving utilizing Bash shell, Intellij integrated development"
template += " environment (IDE), Eclipse integrated development environment (IDE), Toad and SQL developer in the"
template += " process of project-related responsibilities that include writing code in SQL and debugging code to solve"
template += " defects"
template += "\nOne (1) year of related work experience involving analyzing and documenting current-state systems,"
template += " processes, and environments to support project design of future-state systems, processes, and"
template += " environments using Confluent KAFKA, Cloudera Hadoop, Ofsaa, Arcadia Data, Python, Unix, PL/SQL, and SQL"
template += "\nOne (1) year of related work experience involving supporting and performing unit testing to ensure"
template += " successful design and implementation of project solutions"
template += "\nOne (1) year of related work experience involving designing and delivering user manuals and training"
template += " for solutions to ensure successful implementation, adoption, and maintenance of data solutions in big"
template += " data/Hive, OFSAA and using concepts in data warehouse, data lake, and data visualization"
template += "\nOne (1) year of related work experience involving supporting business development activities, including"
template += " proposal development and responses to requests for proposals"
template += "\nOne (1) year of related work experience involving utilizing Confluent (KAFKA) KSQL/SQL, Tableau,"
template += " Jupyter Notebook, Oracle and Apache Tools Impala, Spark, Livy, Zookeeper and Scoop for the completion"
template += " of tasks that include writing code in query language to build data pipelines, creating test cases to"
template += " evaluate products, data analysis, data migration, data modeling, data mapping and data reporting"
template += "\n\nHTML STRING: " + html_strs_list[2] + "\n=========\nMINIMUM REQUIREMENTS:"
template += "\nStrong Python experience especially in data engineering/ML for ML based product development"
template += "\nKnowledge on different algorithms and corresponding Python packages e.g. fuzzy match of strings,"
template += " graph algorithm to create connected lists, etc."
template += "\nStrong coding skills in Pandas, Numpy"
template += "\nGood understanding of Pandas groupby, sort, merge, append, assignment, filters, map, apply"
template += "\nStrong knowledge in Python objects, tuples, list, dict, generators, lambda, etc."
# template += "\n\nHTML STRING: " + html_strs_list[3] + "\n=========\nMINIMUM REQUIREMENTS:"
# template += "\nEnthusiasm for troubleshooting, analyzing,and resolving complex problems"
# template += "\nDemonstrable strong problem-solving and communication skills"
# template += "\nPrepared to be an expert performance engineering resource on multiple initiatives of diverse scopes"
# template += "\nHands-on experience in designing, developing and implementing state of the art test simulation, analysis"
# template += " tools and technologies to ensure platforms deliver industry-leading performance for high availability and"
# template += " great performance for achieving targeting revenues to the clients"
# template += "\nExperience with load testing using JMeter, API, and Microservice testing using RestAssure"
# template += "\nDemonstrable ability to design and delivered performance Testing and Engineering frameworks for complex"
# template += " enterprise applications."
# template += "\nHas played an architect-level role in handling end-to-end (frontend, Middleware and backend systems)"
# template += " performance tuning and optimization of the platform for at least 2 to 3 large engagements"
template += "\n\nHTML STRING: {html_str}\n=========\nMINIMUM REQUIREMENTS:"
prompt = PromptTemplate(
    input_variables=['html_str'],
    template=template,
)
chain = LLMChain(llm=llm, prompt=prompt)
print(f'"{html_strs_list[4]}"')
reqs_list = chain.run(html_strs_list[4]).split('\n')
for req_str in reqs_list:
    if req_str: print('\n'+req_str)

In [ ]:

print()
for html_str in html_strs_list[3].split(' * '):
    print(f'template += "\\n{html_str}"')